In [ ]:
!pip install PyPDF2
!pip install pytesseract
!apt-get install tesseract-ocr

!pip install pillow
!pip install pdf2image
!apt-get install poppler-utils
!pip install pymupdf
!sudo apt-get install tesseract-ocr-vie  # Install Vietnamese language pack for Tesseract
!pip install PyPDF2 pytesseract opencv-python python-docx underthesea



In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
import os
import pandas as pd
import gspread


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)


from google.colab import drive
import os
import re
import PyPDF2
import pytesseract
from PIL import Image
from pdf2image import convert_from_path



import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io


from underthesea import text_normalize
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')




In [ ]:
# Path to the folder in Google Drive
folder_path = '/content/drive/MyDrive/Projects/PTS/Test OCR'

# Get all file names in the folder
file_names = [file for file in os.listdir(folder_path) if file.endswith('.PDF')]

df = pd.DataFrame(file_names, columns=['File Name'])


# Get all PDF file names in the folder
pdf_files = [file for file in os.listdir(folder_path) if file.endswith('.PDF')]
print (pdf_files)


# # Open the Google Sheet (replace 'Sheet Name' with the actual sheet name)
# sheet_name = 'PDF Data'
# worksheet = gc.open(sheet_name).sheet1

# # Convert the DataFrame to a list of lists for writing to the sheet
# data = df.values.tolist()

# # Write the data to the sheet
# worksheet.update('A1', data)

# print("Data successfully added to the Google Sheet.")

['PPS-FIN-PR01.PDF', 'TEST-PPS-FIN-PR01.PDF']


In [ ]:

# Specify your folder ID here
folder_id = '1xMX3Yk7YtkYO7lYHL6rLEMtk36a4LGKR'  # Replace with your folder ID
# Function to perform OCR on a single page
def ocr_page(page):
    pix = page.get_pixmap()
    img = Image.open(io.BytesIO(pix.tobytes()))
    text = pytesseract.image_to_string(img, lang='vie')
    underthesea_normalized_text = text_normalize(text)
    return underthesea_normalized_text

# Function to perform OCR on the entire document
def ocr_document(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += ocr_page(page) + '\n'
    return text



def convert_pdf_text (file_name):
  file_path = os.path.join(folder_path, file_name)

  # Convert each page of the PDF to an image and perform OCR
  pages = convert_from_path(file_path)

  extracted_text = ocr_document(file_path)

  # Print the extracted text
  return (extracted_text)

def save_to_file (file_name, extracted_text):
  drive = GoogleDrive(gauth)

  # Create & upload a text file in the specified folder.
  if 'PDF' in file_name:
    title = file_name.replace('PDF', 'text')
  else:
    title = file_name

  file_metadata = {
      'title': title,
      'parents': [{'id': folder_id}]
  }
  uploaded = drive.CreateFile(file_metadata)
  uploaded.SetContentString(extracted_text)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [ ]:
for file in pdf_files:
  print (file)


PPS-FIN-PR01.PDF
TEST-PPS-FIN-PR01.PDF


In [ ]:
file_name = input("Enter the title for your file: ")

extracted_text = convert_pdf_text(file_name)


Enter the title for your file: TEST-PPS-FIN-PR01.PDF


In [ ]:
save_to_file(file_name, extracted_text)


Uploaded file with ID 1dhDPEA-pOjQElDoy5W40ukZUPagAnShI


**Testing Text Transformation with CHAT GPT API**